In [1]:
import torch
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
from PIL import Image, ImageDraw
import json

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
model.to(device)

AutoShape(
  (model): Model(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (co

In [6]:
dictionary ={
}

In [7]:
def draw_box(img, xmin, ymin, xmax, ymax):
  shape = [(xmin, ymin), (xmax, ymax)]  
# create rectangle image
  img1 = ImageDraw.Draw(img)  
  img1.rectangle(shape, outline ="blue")
  return img 

In [8]:
def draw_boxes(img, results,id):
  for i in range(results.xyxy[0].size()[0]):
    xmin = results.xyxy[0][i][0].cpu().tolist()
    ymin = results.xyxy[0][i][1].cpu().tolist()
    xmax = results.xyxy[0][i][2].cpu().tolist()
    ymax = results.xyxy[0][i][3].cpu().tolist()
    conf = results.xyxy[0][i][4].cpu().tolist()
    class_ = results.xyxy[0][i][5].cpu().tolist()
# class = 2 is the car label class 
    if class_ == 2 and conf >= 0.73:
      id = id+1
      img = draw_box(img, xmin, ymin, xmax, ymax)
      new_object ={
    "Object"+str(id): "["+str(xmin)+","+str(ymin)+","+str(xmax)+","+str(ymax)+"]"}
      dictionary.update(new_object)
  return img, id

In [9]:
import cv2
import numpy as np

In [10]:
frames = []

In [11]:
id = 1 
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('/content/Cars On Highway_.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    results = model(frame)
    frame = Image.fromarray(frame)
    img, id = draw_boxes(frame, results,id)
    frames.append(img)

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()
# Closes all the frames
cv2.destroyAllWindows()

In [12]:
# results in the json file 
json_object = json.dumps(dictionary, indent = 2)
with open("results.json", "w") as outfile:
    outfile.write(json_object)

In [13]:
def write_video(file_path, frames, fps):

    w, h = frames[0].size
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    writer = cv2.VideoWriter(file_path, fourcc, fps, (w, h))

    for frame in frames: 
        cv_image = np.array(frame) 
        writer.write(cv_image)

    writer.release() 

In [14]:
write_video('/content/output_cars_withBoxes.mp4',frames,30)